# Port Scanner mit Python


In [1]:
import socket

start_port = 0
end_port = 150
dst_host = "127.0.0.1"
verbose = False

open_ports = []
for testport in range(start_port, end_port+1):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(0.5)
    try:
        sock.connect((dst_host, int(testport)))
        open_ports.append(testport)
        print(f"Offener Port: {testport}")
        sock.shutdown(socket.SHUT_RD)

    except Exception as e:
        if verbose == 1: print(f"Geschlossener Port: {testport} Expection: {e}")
    
    finally:
        sock.close()

open_ports

Offener Port: 135


[135]

## 1. Inputs <br>
### Helper-Input Funktion für Eingabe von: <br>
Host-IP -> dst_host <br>
Erster Port -> start_port <br>
Letzter Port -> end_port<br>
Ausgabetyp -> verbose

def get_input(typ: type, text: str, std_value: any):
    var = input(text)
    
    if not var:
        return std_value
    if typ != str:
        try:
            return typ(var)
        except ValueError:
            print(f"Falsche Eingabe! Geben Sie eine etwas vom Typ {typ} ein.")
            return get_input(typ, text, std_value)
    else:
        return var

In [2]:
from helper import get_input

dst_host = "127.0.0.1"
start_port = 0
end_port = 1025

print("Input startet... (keine Eingabe = Standart Werte)")
dst_host = get_input(str, f"Gebe die Host-IP ein({dst_host}): ", dst_host)
start_port = get_input(int, f"Gebe den Start Port ein({start_port}): ", start_port)
end_port = get_input(int, f"Gebe den End Port ein({end_port}): ", end_port)

# Ausgabe
(dst_host, start_port, end_port)

Input startet... (keine Eingabe = Standart Werte)


('127.0.0.1', 0, 1025)

## 2. Port Scan
### find_all_async() Funktion aus port_scanner.py

In [3]:
from concurrent import futures
import socket

# Checkt ob ein TCP-Port auf einem Host-System erreichbar/offen ist.
def port_checker(dst_host=str, testport=int, verbose=False) -> tuple[bool, int]:
    print(f'Port Checker on {testport} started...')

    port_open = False

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(0.5)
    val = sock.connect_ex((dst_host, testport))
    
    if val == 0:
        port_open = True
        print(f"Offener Port: {testport}")
    sock.close()

    if verbose and not port_open: print(f"Geschlossener Port: {testport}")

    print(f'Port Checker on {testport} finished...')
    return port_open, testport


# Asynchrone Funktionen brauchen eine Funktion, die den Aufruf managed
# -> hier find_all_async()
open_ports=[]

try:
    # sock.connect() versucht eine Verbindung mit der angegebenen IP/Port herzustellen
    hostinfo = socket.gethostbyaddr(dst_host)
    print(f"Hostname: {hostinfo[0]}, Aliaslist: {hostinfo[1]}, Ipaddrlist: {hostinfo[2]}")
    # wird keine Exception ausgelöst geht es hier weiter
    # close() schließt Verbindung und beendet Socket

    # Erzeugung der Asynchronen Prozesse https://docs.python.org/3/library/concurrent.futures.html
    with futures.ThreadPoolExecutor(max_workers=2000) as executor:
        # erzeugt eine Liste mit den Typen list[Future[tuple[bool, int]]] , also wird der Wert noch erwartet, da die Werte
        # vom Typ Future sind.
        results = [
            executor.submit(port_checker, dst_host=dst_host, testport=i) for i in range(start_port, end_port+1)
        ]
        # Sage dem executor das gewartet werden soll, bis ein Ergebnis zurückkommt
        executor.shutdown(wait=True, cancel_futures=False)
        
        # Wenn thread fertig ist wird der Port der Liste open_ports hinzugefügt
        # muss nicht in genauer Reihenfolge passieren
        for thread in futures.as_completed(results):
            # checken ob der Port offen ist -> Übergebene Bool-Variable
            if thread.result()[0]: open_ports.append(thread.result()[1])

        
#Exception wenn Host nicht erreichbar ist.
except Exception as e:
    print(f"Der Host existiert nicht. Exception: {e}")


# Sortiere Ports für aufsteigende Auflistung
open_ports.sort()
# Gebe sortierte Ports aus
print(f'open_ports returned: {open_ports}')


Hostname: DESKTOP-IRDBMRO, Aliaslist: [], Ipaddrlist: ['127.0.0.1']
Port Checker on 0 started...
Port Checker on 1 started...
Port Checker on 0 finished...
Port Checker on 2 started...
Port Checker on 3 started...
Port Checker on 4 started...
Port Checker on 5 started...
Port Checker on 6 started...
Port Checker on 7 started...
Port Checker on 8 started...
Port Checker on 9 started...
Port Checker on 10 started...
Port Checker on 11 started...
Port Checker on 12 started...
Port Checker on 13 started...
Port Checker on 14 started...
Port Checker on 15 started...
Port Checker on 16 started...
Port Checker on 17 started...
Port Checker on 18 started...
Port Checker on 19 started...
Port Checker on 20 started...
Port Checker on 21 started...
Port Checker on 22 started...
Port Checker on 23 started...
Port Checker on 24 started...
Port Checker on 25 started...
Port Checker on 26 started...
Port Checker on 27 started...
Port Checker on 28 started...
Port Checker on 29 started...
Port Checker

## 3. Services Dict
### get_services_dict() Funktion aus helper.py

nameserver         42/tcp    name                   #Host Name Server
nameserver         42/udp    name                   #Host Name Server
nicname            43/tcp    whois
domain             53/tcp                           #Domain Name Server
domain             53/udp                           #Domain Name Server
bootps             67/udp    dhcps                  #Bootstrap Protocol Server
bootpc             68/udp    dhcpc                  #Bootstrap Protocol Client
tftp               69/udp                           #Trivial File Transfer
gopher             70/tcp
finger             79/tcp
http               80/tcp    www www-http           #World Wide Web
hosts2-ns          81/tcp                           #HOSTS2 Name Server
hosts2-ns          81/udp                           #HOSTS2 Name Server
kerberos           88/tcp    krb5 kerberos-sec      #Kerberos
kerberos           88/udp    krb5 kerberos-sec      #Kerberos
hostname          101/tcp    hostnames              #NIC Host Name Server
iso-tsap          102/tcp                           #ISO-TSAP Class 0
rtelnet           107/tcp                           #Remote Telnet Service
pop2              109/tcp    postoffice             #Post Office Protocol - Version 2
pop3              110/tcp                           #Post Office Protocol - Version 3
sunrpc            111/tcp    rpcbind portmap        #SUN Remote Procedure Call
sunrpc            111/udp    rpcbind portmap        #SUN Remote Procedure Call
auth              113/tcp    ident tap              #Identification Protocol
uucp-path         117/tcp
sqlserv           118/tcp                           #SQL Services
nntp              119/tcp    usenet                 #Network News Transfer Protocol
ntp               123/udp                           #Network Time Protocol
epmap             135/tcp    loc-srv                #DCE endpoint resolution
epmap             135/udp    loc-srv                #DCE endpoint resolution
netbios-ns        137/tcp    nbname                 #NETBIOS Name Service
netbios-ns        137/udp    nbname                 #NETBIOS Name Service
netbios-dgm       138/udp    nbdatagram             #NETBIOS Datagram Service
netbios-ssn       139/tcp    nbsession              #NETBIOS Session Service
imap              143/tcp    imap4                  #Internet Message Access Protocol

In [4]:
# Öffne Datei und speichere sie in Variable
fileread = open("C:\Windows\System32\drivers\etc\services", "r").read()
# Erstelle eine Liste der einzelnen Zeilen der Services-Datei
lines_string = fileread.splitlines()

# erstelle das Dictionary mit {port: service}
services_dict = {}
# Laufe durch alle Elemente in lines_string
for line in lines_string:
    # Splite line beim Hashtag
    split_by_hashtag = line.split(sep="#", maxsplit=2)[0]
    # Splite neue Line in 3 Teile bei Leerzeichen
    split_in_3 = split_by_hashtag.split(maxsplit=3)
    if len(split_in_3) > 1:
        # Trenne zweite Spalte bei "/" auf
        split_port_protc = split_in_3[1].split(sep="/")

        # es wird getestet ob bereits ein Eintrag für den Port existiert
        # https://stackoverflow.com/questions/68193407/how-do-i-check-whether-an-open-port-is-tcp-or-udp
        if int(split_port_protc[0]) not in services_dict.keys():
            # trage key, value par in Dictionary ein
            services_dict[int(split_port_protc[0])] = split_in_3[0]

services_dict

{7: 'echo',
 9: 'discard',
 11: 'systat',
 13: 'daytime',
 17: 'qotd',
 19: 'chargen',
 20: 'ftp-data',
 21: 'ftp',
 22: 'ssh',
 23: 'telnet',
 25: 'smtp',
 37: 'time',
 39: 'rlp',
 42: 'nameserver',
 43: 'nicname',
 53: 'domain',
 67: 'bootps',
 68: 'bootpc',
 69: 'tftp',
 70: 'gopher',
 79: 'finger',
 80: 'http',
 81: 'hosts2-ns',
 88: 'kerberos',
 101: 'hostname',
 102: 'iso-tsap',
 107: 'rtelnet',
 109: 'pop2',
 110: 'pop3',
 111: 'sunrpc',
 113: 'auth',
 117: 'uucp-path',
 118: 'sqlserv',
 119: 'nntp',
 123: 'ntp',
 135: 'epmap',
 137: 'netbios-ns',
 138: 'netbios-dgm',
 139: 'netbios-ssn',
 143: 'imap',
 150: 'sql-net',
 156: 'sqlsrv',
 158: 'pcmail-srv',
 161: 'snmp',
 162: 'snmptrap',
 170: 'print-srv',
 179: 'bgp',
 194: 'irc',
 213: 'ipx',
 322: 'rtsps',
 349: 'mftp',
 389: 'ldap',
 443: 'https',
 445: 'microsoft-ds',
 464: 'kpasswd',
 500: 'isakmp',
 507: 'crs',
 512: 'exec',
 513: 'login',
 514: 'cmd',
 515: 'printer',
 517: 'talk',
 518: 'ntalk',
 520: 'efs',
 522: 'ulp',


## 4. Logs erzeugen

In [5]:
from helper import create_file_name
import logging 

# Erzeuge Logger und dessen File
file_name = create_file_name()
logging.basicConfig(filename=file_name, encoding='utf-8', level=logging.DEBUG)

In [6]:
from datetime import datetime

file_name = datetime.now().strftime("%H:%M:%S")
logging.info(f"""   Zeitstempel: {file_name}, ANGRIFF auf: {dst_host}""")

# Nur TCP als Protokoll, wegen socket library
# Ordne offenen Ports einem Service zu und schreibe in die Logger Datei
for port in open_ports:
    try:
        logging.info(f"   Offener Port: {port}, Protokoll: tcp, Service: {services_dict.get(port)}")
    except TypeError as e:
        logging.error(f"   Port {port} offen, aber nicht in Protokoll: {e}")

print(f"Logs wurden unter dem Namen {file_name} erstellt")



Logs wurden unter dem Namen 13:32:07 erstellt


 ## 5. Fazit (Vergleich zu fertigen Scannern)

 Persönliches Programm:
 - individuell erweiterbar/anpassbar (Logger,...) 
 - Daten bleiben geheim 
 - Programmierkenntnisse nötig 
 - vertrauenswürdig

 Öffentliche Portscanner:
 - Benutzerfreundlich (wenig bis keine Kenntnisse notwendig)
 - Daten werden evtl. weitergegeben 
 - Oft sehr viele Infos --> unübersichtlich 
 - mögliche Gefahren (Malware --> Spyware)